<h2 style="background-color:DodgerBlue; color:white" >Diffrent Validation Schema for multilabel</h2>
<h1>Bonus <span class="label label-default">hold out set for validation</span></h1>
<h1>1 : <span class="label label-default">Custom created k fold cross validation </span></h1>
<h1>2 : <span class="label label-default">MultilabelStratifiedKFold</span></h1>
<h1>3 : <span class="label label-default">RepeatedMultilabelStratifiedKFold</span></h1>
<h1>4 : <span class="label label-default">MultilabelStratifiedShuffleSplit</span></h1>

# load modules and data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.random.seed = 45
import sklearn
import warnings
warnings.filterwarnings("ignore")
!pip install iterative-stratification
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
train_targets_scored = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")

# freature engineering 

In [ ]:
COLS = ['cp_type','cp_dose']
FE = []
for col in COLS:
    for mod in train_features[col].unique():
        FE.append(mod)
        train_features[mod] = (train_features[col] == mod).astype(int)
del train_features['sig_id']
del train_features['cp_type']
del train_features['cp_dose']
FE+=list(train_features.columns) 
del train_targets_scored['sig_id']

# Data to numpy arrays

In [ ]:
X = np.array(train_features.to_numpy(), dtype=np.float)
y = np.array(train_targets_scored.to_numpy(), dtype=np.float)

# hold out set for validation 

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split
X_train, y_train, X_val, y_val = iterative_train_test_split(X, y, test_size = 0.1)
X_train.shape,y_train.shape,X_val.shape,y_val.shape

In [ ]:
print(X_val.shape,y_val.shape)
print(X_val[:2])
y_val[:1]

# k fold cross validation 


In [ ]:
k=4

In [ ]:
for i in range(k):
    np.random.shuffle(X_train)
    np.random.shuffle(y_train)
    X_, y_, X_test, y_test = iterative_train_test_split(X_train,y_train, test_size = 0.2)
    print(X_.shape,y_.shape,X_test.shape,y_test.shape)
    print("model.fit(X_,y_)")
    print("model log loss evaluation for {} fold is {}".format(i+1,np.random.rand(1)))

# model hold out validation

In [ ]:
print("hold out data of shape x: {} y: {} model log loss evaluation is {}".format(X_val.shape,y_val.shape,np.random.rand(1)))

# MultilabelStratifiedKFold

Multilabel stratified K-Folds cross-validator
    Provides train/test indices to split multilabel data into train/test sets.
    This cross-validation object is a variation of KFold that returns
    stratified folds for multilabel data. The folds are made by preserving
    the percentage of samples for each label.

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np


mskf = MultilabelStratifiedKFold(n_splits=2, shuffle=True)

for train_index, test_index in mskf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

# RepeatedMultilabelStratifiedKFold

Repeats Mulilabel Stratified K-Fold n times with different randomization
    in each repetition.

In [ ]:
from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold

rmskf = RepeatedMultilabelStratifiedKFold(n_splits=2,n_repeats=2, random_state=0)

for train_index, test_index in rmskf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

# MultilabelStratifiedShuffleSplit

Multilabel Stratified ShuffleSplit cross-validator
    Provides train/test indices to split data into train/test sets.
    This cross-validation object is a merge of MultilabelStratifiedKFold and
    ShuffleSplit, which returns stratified randomized folds for multilabel
    data. The folds are made by preserving the percentage of each label.
    Note: like the ShuffleSplit strategy, multilabel stratified random splits
    do not guarantee that all folds will be different, although this is
    still very likely for sizeable datasets.

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

msss  = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.5, random_state=0)

for train_index, test_index in msss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)